In [ ]:
from google.colab import files
import io
uploaded = files.upload()

Saving test_data15variables_dropFirst_stratified.csv to test_data15variables_dropFirst_stratified.csv
Saving train_data15variables_dropFirst_stratified.csv to train_data15variables_dropFirst_stratified.csv


In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv(io.BytesIO(uploaded['train_data15variables_dropFirst_stratified.csv']))

In [ ]:
df_train.head()

,catu_2,catu_3,catu_4,catv_2,catv_3,catv_4,catv_5,obsm_2,obsm_3,obsm_4,...,hour_cat_2.0,hour_cat_3.0,catr_2,catr_3,catr_4,catr_5,choc_2,choc_3,choc_4,grav
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,2
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,2
2,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,True,False,False,True,False,4
3,True,False,False,False,True,False,False,False,True,False,...,False,True,False,True,False,False,False,True,False,2
4,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,True,False,True,False,False,1


In [ ]:
X_train = df_train.drop(columns=['grav'])

In [ ]:
y_train = df_train["grav"]

In [ ]:
df_test = pd.read_csv(io.BytesIO(uploaded['test_data15variables_dropFirst_stratified.csv']))

In [ ]:
X_test = df_test.drop(columns=['grav'])

In [ ]:
y_test = df_test["grav"]

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, roc_auc_score, accuracy_score

In [ ]:
train_pool = Pool(X_train, y_train)

In [ ]:
param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'depth': [4, 6, 8],
    'iterations': [100, 200, 300]
}

In [ ]:
scorer = make_scorer(recall_score, average = "macro", labels = [2])

In [ ]:
catboost = CatBoostClassifier()

In [ ]:
grid_search = GridSearchCV(estimator=catboost, param_grid=param_grid, cv=5, scoring=scorer)

In [ ]:
grid_search.fit(X_train,y_train)

In [ ]:
print("Best parameters:", grid_search.best_params_)

Best parameters: {'depth': 4, 'iterations': 200, 'learning_rate': 0.001}


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_train, y_train)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print("Classification Report sur l'ensemble de test avec CatboostClassifier(): \n", classification_report(y_test, y_pred))

Classification Report sur l'ensemble de test avec CatboostClassifier(): 
               precision    recall  f1-score   support

           1       0.57      0.62      0.60      6163
           2       0.38      0.70      0.49      6163
           3       0.41      0.12      0.18      6164
           4       0.41      0.31      0.35      6163

    accuracy                           0.44     24653
   macro avg       0.44      0.44      0.40     24653
weighted avg       0.44      0.44      0.40     24653



In [ ]:
#ADABOOST

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
adaboost = AdaBoostClassifier()


In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.01, 0.001]
}

In [ ]:
scorer = make_scorer(recall_score, average = "macro", labels = [2])

In [ ]:
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring= scorer)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=AdaBoostClassifier(),
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'n_estimators': [50, 100, 200]},
             scoring=make_scorer(recall_score, average=macro, labels=[2]))

In [ ]:
print("Best parameters:", grid_search.best_params_)

Best parameters: {'learning_rate': 0.001, 'n_estimators': 50}


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=0.001)

In [ ]:
print("Classification Report sur l'ensemble de test avec Adaboost(): \n", classification_report(y_test, y_pred))

Classification Report sur l'ensemble de test avec Adaboost(): 
               precision    recall  f1-score   support

           1       0.57      0.62      0.60      6163
           2       0.38      0.70      0.49      6163
           3       0.41      0.12      0.18      6164
           4       0.41      0.31      0.35      6163

    accuracy                           0.44     24653
   macro avg       0.44      0.44      0.40     24653
weighted avg       0.44      0.44      0.40     24653



In [ ]:
# SGD

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier(loss='log')


In [ ]:
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'penalty': ['l1', 'l2'],
    'max_iter': [1000, 2000, 3000],
    'learning_rate': ['constant', 'optimal', 'adaptive']
}

In [ ]:
scorer = make_scorer(recall_score, average = "macro", labels = [2])

In [ ]:
grid_search = GridSearchCV(estimator=sgd, param_grid=param_grid, cv=5, scoring=scorer)

In [ ]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skle

GridSearchCV(cv=5, estimator=SGDClassifier(loss='log'),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'learning_rate': ['constant', 'optimal', 'adaptive'],
                         'max_iter': [1000, 2000, 3000],
                         'penalty': ['l1', 'l2']},
             scoring=make_scorer(recall_score, average=macro, labels=[2]))

In [ ]:
print("Best parameters:", grid_search.best_params_)

Best parameters: {'alpha': 0.1, 'learning_rate': 'optimal', 'max_iter': 2000, 'penalty': 'l2'}


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


SGDClassifier(alpha=0.1, loss='log', max_iter=2000)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
print("Classification Report sur l'ensemble de test avec SGD(): \n", classification_report(y_test, y_pred))

Classification Report sur l'ensemble de test avec SGD(): 
               precision    recall  f1-score   support

           1       0.54      0.71      0.61      6163
           2       0.46      0.65      0.54      6163
           3       0.40      0.17      0.24      6164
           4       0.45      0.37      0.41      6163

    accuracy                           0.48     24653
   macro avg       0.46      0.48      0.45     24653
weighted avg       0.46      0.48      0.45     24653



In [ ]:
#HGB

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
hgb = HistGradientBoostingClassifier()

In [ ]:
param_grid = {
    'max_iter': [50, 100, 200],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [None, 5, 10]
}

In [ ]:
scorer = make_scorer(recall_score, average = "macro", labels = [2])

In [ ]:
grid_search = GridSearchCV(estimator=hgb, param_grid=param_grid, cv=5, scoring=scorer)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [None, 5, 10],
                         'max_iter': [50, 100, 200]},
             scoring=make_scorer(recall_score, average=macro, labels=[2]))

In [ ]:
print("Best parameters:", grid_search.best_params_)

Best parameters: {'learning_rate': 0.01, 'max_depth': 10, 'max_iter': 200}


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_train, y_train)

HistGradientBoostingClassifier(learning_rate=0.01, max_depth=10, max_iter=200)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
print("Classification Report sur l'ensemble de test avec HGB(): \n", classification_report(y_test, y_pred))

Classification Report sur l'ensemble de test avec HGB(): 
               precision    recall  f1-score   support

           1       0.64      0.74      0.69      6163
           2       0.52      0.66      0.59      6163
           3       0.41      0.28      0.33      6164
           4       0.48      0.43      0.46      6163

    accuracy                           0.53     24653
   macro avg       0.51      0.53      0.52     24653
weighted avg       0.51      0.53      0.52     24653



In [ ]:
# Bagging Classifier (avec choix des weak models)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
base_estimators = [
    (LogisticRegression(max_iter=1000), {'C': 0.001, 'penalty': 'l2'}).fit(X_train,y_train),
    (SVC(),{'C': 0.1, 'kernel': 'linear'}).fit(X_train,y_train),
    (KNeighborsClassifier(), {'n_neighbors' : 17}).fit(X_train,y_train)
]

AttributeError: 'tuple' object has no attribute 'fit'

In [ ]:
bagging = BaggingClassifier(base_estimators)

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0]
}

In [ ]:
scorer = make_scorer(recall_score, average = "macro", labels = [2])

In [ ]:
grid_search = GridSearchCV(estimator=BaggingClassifier(base_estimators), param_grid=param_grid, cv=5, scoring=scorer, error_score='raise')

In [ ]:
grid_search.fit(X_train, y_train)

InvalidParameterError: The 'estimator' parameter of BaggingClassifier must be an object implementing 'fit' and 'predict' or None. Got [(LogisticRegression(max_iter=1000), {'C': 0.001, 'penalty': 'l2'}), (SVC(), {'C': 0.1, 'kernel': 'linear'}), (KNeighborsClassifier(), {'n_neighbors': 17})] instead.

In [ ]:
# Bagging Classifier avec base model par default (Decision Tree)

In [ ]:
bagging = BaggingClassifier()

In [ ]:
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}


In [ ]:
scorer = make_scorer(recall_score, average = "macro", labels = [2])

In [ ]:
grid_search = GridSearchCV(estimator=bagging, param_grid=param_grid, cv=5, scoring=scorer)


In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BaggingClassifier(),
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'max_features': [0.5, 0.7, 1.0],
                         'max_samples': [0.5, 0.7, 1.0],
                         'n_estimators': [10, 50, 100]},
             scoring=make_scorer(recall_score, average=macro, labels=[2]))

In [ ]:
print("Best parameters:", grid_search.best_params_)

Best parameters: {'bootstrap': False, 'bootstrap_features': True, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 100}


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_train, y_train)

BaggingClassifier(bootstrap=False, bootstrap_features=True, max_features=0.5,
                  max_samples=0.5, n_estimators=100)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
print("Classification Report sur l'ensemble de test avec BaggingClassifier(): \n", classification_report(y_test, y_pred))

Classification Report sur l'ensemble de test avec BaggingClassifier(): 
               precision    recall  f1-score   support

           1       0.59      0.78      0.67      6163
           2       0.52      0.64      0.57      6163
           3       0.43      0.27      0.33      6164
           4       0.49      0.39      0.43      6163

    accuracy                           0.52     24653
   macro avg       0.51      0.52      0.50     24653
weighted avg       0.51      0.52      0.50     24653



In [ ]:
## Stacking


In [ ]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_regression = LogisticRegression(max_iter=1000, C=0.001, penalty='l2')

In [ ]:
SVM = SVC(C=0.1, kernel='linear')

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=17)

In [ ]:
base_estimators = [
    ('logistic_regression', logistic_regression),
    ('svm', SVM),
    ('knn', knn_model)
]

In [ ]:
stacking = StackingClassifier(estimators=base_estimators, final_estimator=RandomForestClassifier())

In [ ]:
param_grid = {
    'final_estimator__n_estimators': [10, 50, 100],
    'final_estimator__max_depth': [None, 5, 10],
    'final_estimator__min_samples_split': [2, 5, 10]
}

In [ ]:
scorer = make_scorer(recall_score, average = "macro", labels = [2])

In [ ]:
grid_search = GridSearchCV(estimator=stacking, param_grid=param_grid, cv=5, scoring=scorer, error_score = "raise")

In [ ]:
grid_search.fit(X_train, y_train)